In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use(['dark_background'])
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

print('Train:', df_train.shape)
print('Test:', df_test.shape)

Train: (3000, 23)
Test: (4398, 22)


In [3]:
df_train.columns

Index(['id', 'belongs_to_collection', 'budget', 'genres', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'runtime', 'spoken_languages',
       'status', 'tagline', 'title', 'Keywords', 'cast', 'crew', 'revenue'],
      dtype='object')

# NaN

## Train

In [4]:
nan_series = df_train.isna().sum()
nan_df = pd.DataFrame({'features': nan_series.index, 'total': nan_series.values})
nan_df['perc'] = nan_df['total'] / df_train.shape[0]
nan_df.sort_values(['perc'], ascending = False)

,features,total,perc
1,belongs_to_collection,2396,0.798667
4,homepage,2054,0.684667
17,tagline,597,0.199000
19,Keywords,276,0.092000
11,production_companies,156,0.052000
12,production_countries,55,0.018333
15,spoken_languages,20,0.006667
21,crew,16,0.005333
20,cast,13,0.004333
8,overview,8,0.002667


In [5]:
drop_cols = ['belongs_to_collection', 'homepage', # many NaNs
             'overview', 'tagline', 'title', 'original_title', # NLP
             'poster_path', 'imdb_id'] # unused

df_train.drop(drop_cols, axis = 1, inplace = True)
df_test.drop(drop_cols, axis = 1, inplace = True)

print('Train:', df_train.shape)
print('Test:', df_test.shape)

Train: (3000, 15)
Test: (4398, 14)


In [6]:
nan_series = df_train.isna().sum()
nan_df = pd.DataFrame({'features': nan_series.index, 'total': nan_series.values})
nan_df['perc'] = nan_df['total'] / df_train.shape[0]
nan_df.sort_values(['perc'], ascending = False)

,features,total,perc
11,Keywords,276,0.092000
5,production_companies,156,0.052000
6,production_countries,55,0.018333
9,spoken_languages,20,0.006667
13,crew,16,0.005333
12,cast,13,0.004333
2,genres,7,0.002333
8,runtime,2,0.000667
0,id,0,0.000000
1,budget,0,0.000000


In [7]:
df_train['Keywords'].fillna('[]', inplace = True)
df_train['production_companies'].fillna('[]', inplace = True)
df_train['production_countries'].fillna('[]', inplace = True)
df_train['spoken_languages'].fillna('[]', inplace = True)
df_train['crew'].fillna('[]', inplace = True)
df_train['cast'].fillna('[]', inplace = True)
df_train['genres'].fillna('[]', inplace = True)

df_train['runtime'].fillna(df_train['runtime'].mean().item(), inplace = True)

df_train.isna().sum()

id                      0
budget                  0
genres                  0
original_language       0
popularity              0
production_companies    0
production_countries    0
release_date            0
runtime                 0
spoken_languages        0
status                  0
Keywords                0
cast                    0
crew                    0
revenue                 0
dtype: int64

## Test

In [8]:
nan_series = df_test.isna().sum()
nan_df = pd.DataFrame({'features': nan_series.index, 'total': nan_series.values})
nan_df['perc'] = nan_df['total'] / df_test.shape[0]
nan_df.sort_values(['perc'], ascending = False)

,features,total,perc
11,Keywords,393,0.089359
5,production_companies,258,0.058663
6,production_countries,102,0.023192
9,spoken_languages,42,0.009550
13,crew,22,0.005002
2,genres,16,0.003638
12,cast,13,0.002956
8,runtime,4,0.000910
10,status,2,0.000455
7,release_date,1,0.000227


In [9]:
df_test['Keywords'].fillna('[]', inplace = True)
df_test['production_companies'].fillna('[]', inplace = True)
df_test['production_countries'].fillna('[]', inplace = True)
df_test['spoken_languages'].fillna('[]', inplace = True)
df_test['crew'].fillna('[]', inplace = True)
df_test['cast'].fillna('[]', inplace = True)
df_test['genres'].fillna('[]', inplace = True)

df_test['runtime'].fillna(df_train['runtime'].mean().item(), inplace = True)
df_test['status'].fillna('Released', inplace = True)
df_test['status'].replace('Post Production', 'Released', inplace = True)

df_test.isna().sum()

id                      0
budget                  0
genres                  0
original_language       0
popularity              0
production_companies    0
production_countries    0
release_date            1
runtime                 0
spoken_languages        0
status                  0
Keywords                0
cast                    0
crew                    0
dtype: int64

In [10]:
df_test[df_test['release_date'].isna()]

,id,budget,genres,original_language,popularity,production_companies,production_countries,release_date,runtime,spoken_languages,status,Keywords,cast,crew
828,3829,0,"[{'id': 18, 'name': 'Drama'}]",en,0.009057,[],[],NaN,90.0,[],Released,[],[],[]


In [11]:
df_test['release_date'].fillna('03/20/01', inplace = True)
df_test.loc[828]

id                                               3829
budget                                              0
genres                  [{'id': 18, 'name': 'Drama'}]
original_language                                  en
popularity                                   0.009057
production_companies                               []
production_countries                               []
release_date                                 03/20/01
runtime                                            90
spoken_languages                                   []
status                                       Released
Keywords                                           []
cast                                               []
crew                                               []
Name: 828, dtype: object

## Merge sets

In [12]:
train_labels = df_train.pop('revenue')
df_full = pd.concat([df_train, df_test], keys = ['train', 'test'])
df_full.shape

(7398, 14)

In [13]:
df_full.head()

id    budget                                             genres  \
train 0   1  14000000                     [{'id': 35, 'name': 'Comedy'}]   
      1   2  40000000  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
      2   3   3300000                      [{'id': 18, 'name': 'Drama'}]   
      3   4   1200000  [{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...   
      4   5         0  [{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...   

        original_language  popularity  \
train 0                en    6.575393   
      1                en    8.248895   
      2                en   64.299990   
      3                hi    3.174936   
      4                ko    1.148070   

                                      production_companies  \
train 0  [{'name': 'Paramount Pictures', 'id': 4}, {'na...   
      1        [{'name': 'Walt Disney Pictures', 'id': 2}]   
      2  [{'name': 'Bold Films', 'id': 2266}, {'name': ...   
      3                                                 []   
      4                                                 []   

                                      production_countries release_date  \
train 0  [{'iso_3166_1': 'US', 'name': 'United States o...      2/20/15   
      1  [{'iso_3166_1': 'US', 'name': 'United States o...       8/6/04   
      2  [{'iso_3166_1': 'US', 'name': 'United States o...     10/10/14   
      3            [{'iso_3166_1': 'IN', 'name': 'India'}]       3/9/12   
      4      [{'iso_3166_1': 'KR', 'name': 'South Korea'}]       2/5/09   

         runtime                                   spoken_languages    status  \
train 0     93.0           [{'iso_639_1': 'en', 'name': 'English'}]  Released   
      1    113.0           [{'iso_639_1': 'en', 'name': 'English'}]  Released   
      2    105.0           [{'iso_639_1': 'en', 'name': 'English'}]  Released   
      3    122.0  [{'iso_639_1': 'en', 'name': 'English'}, {'iso...  Released   
      4    118.0           [{'iso_639_1': 'ko', 'name': '한국어/조선말'}]  Released   

                                                  Keywords  \
train 0  [{'id': 4379, 'name': 'time travel'}, {'id': 9...   
      1  [{'id': 2505, 'name': 'coronation'}, {'id': 42...   
      2  [{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...   
      3  [{'id': 10092, 'name': 'mystery'}, {'id': 1054...   
      4                                                 []   

                                                      cast  \
train 0  [{'cast_id': 4, 'character': 'Lou', 'credit_id...   
      1  [{'cast_id': 1, 'character': 'Mia Thermopolis'...   
      2  [{'cast_id': 5, 'character': 'Andrew Neimann',...   
      3  [{'cast_id': 1, 'character': 'Vidya Bagchi', '...   
      4  [{'cast_id': 3, 'character': 'Chun-soo', 'cred...   

                                                      crew  
train 0  [{'credit_id': '59ac067c92514107af02c8c8', 'de...  
      1  [{'credit_id': '52fe43fe9251416c7502563d', 'de...  
      2  [{'credit_id': '54d5356ec3a3683ba0000039', 'de...  
      3  [{'credit_id': '52fe48779251416c9108d6eb', 'de...  
      4  [{'credit_id': '52fe464b9251416c75073b43', 'de...

# Categorical Features

In [16]:
df_features = df_full[['id', 'budget', 'original_language', 'popularity', 'runtime', 'release_date', 'status']]
df_features.head()

id    budget original_language  popularity  runtime release_date  \
train 0   1  14000000                en    6.575393     93.0      2/20/15   
      1   2  40000000                en    8.248895    113.0       8/6/04   
      2   3   3300000                en   64.299990    105.0     10/10/14   
      3   4   1200000                hi    3.174936    122.0       3/9/12   
      4   5         0                ko    1.148070    118.0       2/5/09   

           status  
train 0  Released  
      1  Released  
      2  Released  
      3  Released  
      4  Released

## Label Encoder

In [20]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df_features['original_language'] = le.fit_transform(df_features['original_language'])
df_features['status'] = le.fit_transform(df_features['status'])

## Time Features

In [21]:
df_features['release_date'] = pd.to_datetime(df_features['release_date'])
df_features['release_year'] = df_features['release_date'].dt.year
df_features['release_month'] = df_features['release_date'].dt.month
df_features['release_quarter'] = df_features['release_date'].dt.quarter
df_features.drop(['release_date'], axis = 1, inplace = True)

df_features.head()

id    budget  original_language  popularity  runtime  status  \
train 0   1  14000000                 10    6.575393     93.0       0   
      1   2  40000000                 10    8.248895    113.0       0   
      2   3   3300000                 10   64.299990    105.0       0   
      3   4   1200000                 16    3.174936    122.0       0   
      4   5         0                 24    1.148070    118.0       0   

         release_year  release_month  release_quarter  
train 0          2015              2                1  
      1          2004              8                3  
      2          2014             10                4  
      3          2012              3                1  
      4          2009              2                1

## Production Companies

In [54]:
def extract_dict(dict_str):
    dict_res = dict_str.replace('[', '').replace(']', '')
    dict_res = dict_res.replace('}, {', '};{')
    dict_res = dict_res.split(';')
    return dict_res

In [88]:
from ast import literal_eval

production_companies = set()

for item in df_full['production_companies']:
    if item == '[]':
        continue
    companies_dict = extract_dict(item)
    for company in companies_dict:
        production_companies.add(literal_eval(company)['name'])
    
len(production_companies)

7087

In [89]:
company_counts = pd.DataFrame(index = production_companies)
company_counts['counts'] = 0

for item in df_full['production_companies']:
    if item == '[]':
        continue
    companies_dict = extract_dict(item)
    for company in companies_dict:
        company_id = literal_eval(company)['name']
        company_counts['counts'].loc[company_id] += 1
        
company_counts.sort_values('counts', ascending = False).head(10)

,counts
Warner Bros.,491
Universal Pictures,463
Paramount Pictures,393
Twentieth Century Fox Film Corporation,341
Columbia Pictures,236
Metro-Goldwyn-Mayer (MGM),207
New Line Cinema,198
Touchstone Pictures,158
Walt Disney Pictures,147
Columbia Pictures Corporation,140


In [90]:
company_counts['perc'] = company_counts['counts'] / len(production_companies)
company_counts.sort_values('perc', ascending = False).to_csv('company_counts.csv')

## Genres

In [74]:
genres = set()

for item in df_full['genres']:
    if item == '[]':
        continue
    genres_dict = extract_dict(item)
    for genre in genres_dict:
        genres.add(literal_eval(genre)['name'])
    
genres

{'Action',
 'Adventure',
 'Animation',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Foreign',
 'History',
 'Horror',
 'Music',
 'Mystery',
 'Romance',
 'Science Fiction',
 'TV Movie',
 'Thriller',
 'War',
 'Western'}

In [78]:
for genre in genres:
    col_name = 'genre_' + genre
    df_features[col_name] = 0
    
df_features.head()

id    budget  original_language  popularity  runtime  status  \
train 0   1  14000000                 10    6.575393     93.0       0   
      1   2  40000000                 10    8.248895    113.0       0   
      2   3   3300000                 10   64.299990    105.0       0   
      3   4   1200000                 16    3.174936    122.0       0   
      4   5         0                 24    1.148070    118.0       0   

         release_year  release_month  release_quarter  genre_Romance  \
train 0          2015              2                1              0   
      1          2004              8                3              0   
      2          2014             10                4              0   
      3          2012              3                1              0   
      4          2009              2                1              0   

         genre_Comedy  genre_Action  genre_War  genre_Music  genre_Crime  \
train 0             0             0          0            0            0   
      1             0             0          0            0            0   
      2             0             0          0            0            0   
      3             0             0          0            0            0   
      4             0             0          0            0            0   

         genre_Foreign  genre_Drama  genre_Fantasy  genre_Mystery  \
train 0              0            0              0              0   
      1              0            0              0              0   
      2              0            0              0              0   
      3              0            0              0              0   
      4              0            0              0              0   

         genre_History  genre_Horror  genre_TV Movie  genre_Adventure  \
train 0              0             0               0                0   
      1              0             0               0                0   
      2              0             0               0                0   
      3              0             0               0                0   
      4              0             0               0                0   

         genre_Thriller  genre_Western  genre_Documentary  \
train 0               0              0                  0   
      1               0              0                  0   
      2               0              0                  0   
      3               0              0                  0   
      4               0              0                  0   

         genre_Science Fiction  genre_Family  genre_Animation  
train 0                      0             0                0  
      1                      0             0                0  
      2                      0             0                0  
      3                      0             0                0  
      4                      0             0                0

In [81]:
count = 0
for index, row in df_full.iterrows():
    count += 1
    if count % 100 == 0:
        print(count, end = ' ')
    for genre in genres:
        col_name = 'genre_' + genre
        if genre in row['genres']:
            df_features[col_name].loc[index] = 1
            
df_features.head()

100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 3400 3500 3600 3700 3800 3900 4000 4100 4200 4300 4400 4500 4600 4700 4800 4900 5000 5100 5200 5300 5400 5500 5600 5700 5800 5900 6000 6100 6200 6300 6400 6500 6600 6700 6800 6900 7000 7100 7200 7300 

id    budget  original_language  popularity  runtime  status  \
train 0   1  14000000                 10    6.575393     93.0       0   
      1   2  40000000                 10    8.248895    113.0       0   
      2   3   3300000                 10   64.299990    105.0       0   
      3   4   1200000                 16    3.174936    122.0       0   
      4   5         0                 24    1.148070    118.0       0   

         release_year  release_month  release_quarter  genre_Romance  \
train 0          2015              2                1              0   
      1          2004              8                3              1   
      2          2014             10                4              0   
      3          2012              3                1              0   
      4          2009              2                1              0   

         genre_Comedy  genre_Action  genre_War  genre_Music  genre_Crime  \
train 0             1             0          0            0            0   
      1             1             0          0            0            0   
      2             0             0          0            0            0   
      3             0             0          0            0            0   
      4             0             1          0            0            0   

         genre_Foreign  genre_Drama  genre_Fantasy  genre_Mystery  \
train 0              0            0              0              0   
      1              0            1              0              0   
      2              0            1              0              0   
      3              0            1              0              0   
      4              0            0              0              0   

         genre_History  genre_Horror  genre_TV Movie  genre_Adventure  \
train 0              0             0               0                0   
      1              0             0               0                0   
      2              0             0               0                0   
      3              0             0               0                0   
      4              0             0               0                0   

         genre_Thriller  genre_Western  genre_Documentary  \
train 0               0              0                  0   
      1               0              0                  0   
      2               0              0                  0   
      3               1              0                  0   
      4               1              0                  0   

         genre_Science Fiction  genre_Family  genre_Animation  
train 0                      0             0                0  
      1                      0             1                0  
      2                      0             0                0  
      3                      0             0                0  
      4                      0             0                0